In [ ]:
import os
import shutil
import tempfile
import pandas as pd
import time
import gc
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import monai
import nibabel as nib
from monai.utils import set_determinism, first
from monai.inferers import sliding_window_inference
from monai.transforms import *

from monai.config import print_config
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric

from monai.data import (
    DataLoader,
    Dataset,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)

from skimage import measure
from sklearn.metrics import roc_auc_score

import torch
import glob
print_config()

In [ ]:
# BG inference

In [ ]:
# dataloader passed!
# Using a custom data path and dataset

In [ ]:
model = monai.networks.nets.BasicUNet(spatial_dims=3, 
                                      in_channels=1,
                                      out_channels=2,
                                      features=(32, 32, 64, 128, 256, 32),
                                     )

In [ ]:
device = "cuda:0"
model.to(device)

In [ ]:
ReLU = torch.nn.ReLU()

def normReLU(input):
    if torch.max(ReLU(input)) != 0:
        output = (ReLU(input)) / (torch.max(ReLU(input)))
    else: output = torch.zeros(input.shape)
    return output

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
model.eval()
# model2.eval()
flag_save_nifti =1
vs_i = np.array([1.,1.,1.])

In [ ]:
with torch.no_grad():
    for step, val_batch in enumerate(valid_loader):
        fname = val_batch["t2_meta_dict"]['filename_or_obj'][0].split('/')[-1]
        dpath = val_batch["t2_meta_dict"]['filename_or_obj'][0].split(fname)[0]
        print(step, dpath)
        pid = val_batch["t2_meta_dict"]['filename_or_obj'][0].split('/')[-2]
        h = nib.load(val_batch["t2_meta_dict"]['filename_or_obj'][0])
        
        t2 = val_batch["t2"].to(device)
        
        with torch.no_grad():
            val_outputs = sliding_window_inference(
                t2, (64, 64, 64), 4, model, overlap=0.7,mode='gaussian')
       

        for b in range(val_outputs.shape[0]):
            for c in range(val_outputs.shape[1]):
                val_outputs[b,c] = normReLU(val_outputs[b,c])

        val_outputs_bimask = (val_outputs[:,1] > 0.42).unsqueeze(1)

        r = val_outputs_bimask[0, 0].cpu().detach().numpy()
        prob = val_outputs[0,1].cpu().detach().numpy()
        if flag_save_nifti == 1:
            hinfo = nib.load(val_batch["t2_meta_dict"]['filename_or_obj'][0])
            h = nib.Nifti1Image(r.astype(np.int16),hinfo.affine)
            nib.save(h,f'{dpath}/DLseg_bg.nii.gz')        

In [ ]:
# PVS inference
# dataloader passed!
# Using a custom data path and dataset

In [ ]:
model = monai.networks.nets.BasicUNet(spatial_dims=3, 
                                      in_channels=1,
                                      out_channels=2,
                                      features=(32, 32, 64, 128, 256, 32),
                                     )

In [ ]:
device = "cuda:0"
model.to(device)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
model.eval()
# model2.eval()
flag_save_nifti =1

In [ ]:
from monai.inferers import sliding_window_inference


with torch.no_grad():
    for step, val_batch in enumerate(valid_loader):
        fname = val_batch["t2_meta_dict"]['filename_or_obj'][0].split('/')[-1]
        dpath = val_batch["t2_meta_dict"]['filename_or_obj'][0].split(fname)[0]
        print(step, dpath)


        pid = val_batch["t2_meta_dict"]['filename_or_obj'][0].split('/')[-2]
        h = nib.load(val_batch["t2_meta_dict"]['filename_or_obj'][0])
        
        t2 = val_batch["t2"].to(device)
        
        with torch.no_grad():
            val_outputs = sliding_window_inference(
                t2, (64, 64, 64), 4, model, overlap=0.55,mode='gaussian')
       
        y_hat_prob = val_outputs.softmax(1)
        y_hat = y_hat_prob.argmax(1)

        if flag_save_nifti == 1:
            hinfo = nib.load(val_batch["t2_meta_dict"]['filename_or_obj'][0])
            h = nib.Nifti1Image(y_hat[0].astype(np.int16),hinfo.affine)
            nib.save(h,f'{dpath}/DLseg_pvs.nii.gz')  

